In [1]:
# Notebook for Monte-Carlo Simulation

In [2]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

import numpy as np
from scipy import optimize
from model import Model
from matplotlib import pyplot as plt
from Simulation import simulate
import EGM
import tools
import DC_EGM
import Estimation as est
import pickle
import seaborn as sns
import auxiliary_funcs as af
import time as time
import copy as copy

import warnings
warnings.filterwarnings('ignore') #:)

In [3]:
par2 = {'a_max':100,
        'Smax': 6,
        'sigma_taste': 0.3,
        'sigma': 0.5,
        'm_initial': 1,
        'phi_high': 5,
        'phi_low': 1,
        'theta_high':0.66,
        'theta_low':0.33,
        'lambda_max':0.797
        }

In [4]:
model = Model()
model.setup()

# solution and simulation specs
model.par.Na = 200
model.par.Tmax = 45
model.par.Tsim = model.par.Tmax
model.par.Ntypes = 4
model.par.N = 10000
model.par.Ns = 10
# set parameters
for key, val in par2.items():
    model.par.__setattr__(key, val)

model.par.easy_par=False

model.set_grids()
par = model.par
sol = model.sol
sim = model.sim

In [5]:
model.par.Ns

10

In [6]:
model.solve()
print("Model Solved")

Model Solved


In [6]:
run = True

if run == True: 
    Results = {}
    dist_true = [0.25,0.25,0.25,0.25]
    N = 2 # Number of MC iterations 

    for i in range(N): 
        par.dist = dist_true 

        est.reset_sim(sim,model)

        seed_obj = np.random
        seed_obj.seed(1667+i)
        par.random = seed_obj

        simulate(sim,sol,par)
        
        data = model.sim

        sec = time.time()

        res,_ = est.estimate(est.obj,data,model,[0.2,0.8])

        time_it = time.time() - sec 

        result = {  "p1" : res.x[0], 
                    "p2" : res.x[1], 
                    "fun" : res.fun, 
                    "Succes" : res.success,
                    "Time" : time_it }
        
        Results[f"result_{i}"] = result

179.3820540163831
172.85722125838384
209.21030150464625
145.98335750866173
124.3301435062383
119.07495079689834
92.77380310902964
71.61603874228811
62.833897884270186
58.18255535297013
95.82912248250894
128.9591739322688
63.24057015729228
80.66420954789123
57.97609954826757
50.80050247853648
46.301024586602736
48.91644986566038
36.71957159521356
28.67563175771719
36.23799866924027
21.943875878175202
20.37413289327236
6.844183355761468
1.9036249955376596
24.15232143178187
11.156427490336899
7.050109074923767
12.537908830250986
4.327461759536842
10.443377293785769
3.027044955660183
1.8813351823009439
5.477703487491807
4.622668595186152
1.6432956208169858
3.793474724186958
1.4469326483534573
2.511793958624781
1.4251311600137186
1.594308300860921
1.399829215109033
1.6959084761928636
1.392811262346751
1.5573054036848466
1.4062489344888414
1.3856112149951614
1.5573054036848466
1.4327184671597468
1.3875802343031618
1.4628258776680103
1.4000126017334296
1.379795018249773
1.403557278184753
1.40

10

In [7]:
Results

{'result_0': {'p1': 0.4692050581666265,
  'p2': 0.5448131926881603,
  'fun': 1.363762617279114,
  'Succes': True,
  'Time': 482.3205873966217},
 'result_1': {'p1': 0.48018889619270344,
  'p2': 0.5385724681243294,
  'fun': 0.35366928909214435,
  'Succes': True,
  'Time': 575.7138378620148}}

In [7]:
run = True

sol_true = copy.deepcopy(sol)

if run == True: 
    Results = {}
    
    dist_true = [0.25,0.25,0.25,0.25]
    phi_high_true = 5

    N = 2 # Number of MC iterations 

    for i in range(N): 
        par.dist = dist_true 

        est.reset_sim(sim,model)

        seed_obj = np.random
        seed_obj.seed(1687+i)
        par.random = seed_obj 

        setattr(model.par, "phi_high", phi_high_true)
        simulate(sim,sol_true,par)
        
        data = model.sim

        sec = time.time()

        res = est.estimate_transfer(data,model,[0.2,0.8,3])

        time_it = time.time() - sec 

        result = {  "p1" : res.x[0], 
                    "p2" : res.x[1],
                    "Phi_high" : res.x[2], 
                    "fun" : res.fun, 
                    "Succes" : res.success,
                    "Time" : time_it }
        
        Results[f"result_{i}"] = result

565.4271421474942
548.595024661684
598.9060051844313
521.9452707441408
493.31691902680393
451.7486466609882
448.9711413663681
398.3669277570734
376.92942496668667
314.97297209419236
291.0387102713189
229.3138073805639
198.79483305098645
151.56586394006615
181.04117646103782
244.0415338253821
184.59067347746554
212.73486376135295
177.05716278984787
146.06899980516695
135.71958171630573
117.44945172325208
96.70976481516726
160.41738256210607
131.8322220584222
98.54720353912205
97.68198745449043
66.76057204779491
48.55677756414344
55.54711335488999
52.72041903354308
29.24983152681707
28.21725716537841
29.424463146960875
51.34070469399349
33.28553301345709
46.747722634228204
33.38145706035139
32.99519959733463
27.362709488701825
35.757758711782984
27.228218556781066
36.3338162916224
23.558395266228224
26.757942701092603
29.679449746471526
24.806519320449972
33.42591699854841
24.64100619568559
25.552727224041703
23.51605552912428
21.000995289214696
20.582548787924154
23.957160536909097
22.0

In [8]:
Results 

{'result_0': {'p1': 0.5038205995825007,
  'p2': 0.5093884640872781,
  'Phi_high': 4.955290133676579,
  'fun': 0.6158972517583441,
  'Succes': True,
  'Time': 926.1521854400635},
 'result_1': {'p1': 0.4869733341432705,
  'p2': 0.486161324258513,
  'Phi_high': 5.148972729343291,
  'fun': 3.1567802741156856,
  'Succes': True,
  'Time': 921.5185072422028}}